# Feb 13, 2025: register the parcellations to ccfv3
for brainrendering

In [1]:
import csv 
import os
import numpy as np
import pandas as pd
import ants
import seaborn as sns
import dill as pickle 

from allensdk.core.mouse_connectivity_cache import (
    MouseConnectivityCache,
    MouseConnectivityApi
)
from allensdk.api.queries.ontologies_api import OntologiesApi

import nrrd
import nibabel as nib
import ants

In [2]:
# ccfv3
atlas_path = f'{os.environ["HOME"]}/new_mouse_dataset/allen_atlas_ccfv3'
mcc_path = f'{atlas_path}/MouseConnectivity'
mcc = MouseConnectivityCache(
    resolution=25, # in micro meters (um)
    ccf_version=MouseConnectivityApi().CCF_2017,
    manifest_file=f'{mcc_path}/manifest.json',
)
AVGT, metaAVGT = mcc.get_template_volume()
ANO, metaANO = mcc.get_annotation_volume()
AVGT = AVGT.astype(np.float32)
ANO = ANO.astype(np.uint32)
print(AVGT.shape, ANO.shape)

STree = mcc.get_structure_tree()
STree_df = pd.DataFrame(STree.nodes()) 

(528, 320, 456) (528, 320, 456)


In [3]:
temp_file = f'{mcc_path}/average_template_25.nrrd'
data, header = nrrd.read(temp_file)

# affine information
spacing = header.get("space directions", np.eye(3))
origin = header.get("space origin", np.zeros(3))

# construct affine matrix for nifti
affine = np.eye(4)
affine[:3, :3] = spacing / 1000
affine[:3, 3] = origin

# convert to nifti
nifti_img = nib.Nifti1Image(data, affine)
nifti_file = f'{mcc_path}/average_template_25.nii.gz'
nib.save(nifti_img, nifti_file)

In [4]:
affine

array([[0.025, 0.   , 0.   , 0.   ],
       [0.   , 0.025, 0.   , 0.   ],
       [0.   , 0.   , 0.025, 0.   ],
       [0.   , 0.   , 0.   , 1.   ]])

In [5]:
ccfv3_img = ants.image_read(nifti_file)
ccfv3_img = ants.reorient_image2(ccfv3_img, orientation='ASR')
ccfv3_img

ANTsImage (ASR)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (320, 456, 528)
	 Spacing    : (0.025, 0.025, 0.025)
	 Origin     : (-13.175, -7.975, 11.375)
	 Direction  : [ 0.  0.  1.  1.  0.  0.  0. -1.  0.]

---

In [6]:
class ARGS():
    pass

args = ARGS()

args.SEED = 100

np.random.seed(args.SEED)

In [7]:
BASE_path = f'{os.environ["HOME"]}/new_mouse_dataset'
PARCELS_path = f'{BASE_path}/parcels'
os.system(f'mkdir -p {PARCELS_path}')

0

In [8]:
args.source = 'allen'
args.space = 'ccfv2'
args.brain_div = 'whl'
args.num_rois = 120
args.resolution = 25 #200, 25 #um

PARC_DESC = (
    f'source-{args.source}'
    f'_space-{args.space}'
    f'_braindiv-{args.brain_div}'
    f'_nrois-{args.num_rois}'
    f'_res-{args.resolution}'
)
PARC_DESC

'source-allen_space-ccfv2_braindiv-whl_nrois-120_res-25'

In [9]:
TEMP_DESC = (
    f'source-{args.source}'
    f'_space-{args.space}'
    f'_res-{args.resolution}'
)
TEMP_DESC

'source-allen_space-ccfv2_res-25'

In [10]:
parcels_img = ants.image_read(f'{PARCELS_path}/{PARC_DESC}_desc-parcels.nii.gz')
parcels_img

ANTsImage (RAI)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (456, 320, 528)
	 Spacing    : (0.025, 0.025, 0.025)
	 Origin     : (-17.155, -15.805, -20.105)
	 Direction  : [1. 0. 0. 0. 1. 0. 0. 0. 1.]

In [11]:
temp_img = ants.image_read(f'{PARCELS_path}/{TEMP_DESC}_desc-template.nii.gz')
temp_img

ANTsImage (RAI)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (456, 320, 528)
	 Spacing    : (0.025, 0.025, 0.025)
	 Origin     : (-17.155, -15.805, -20.105)
	 Direction  : [1. 0. 0. 0. 1. 0. 0. 0. 1.]

---

In [12]:
affine_tx = ants.registration(
    fixed=ccfv3_img,
    moving=temp_img,
    type_of_transform='AffineFast',
    random_seed=args.SEED,
)

In [13]:
temp_img_affine_tx = ants.apply_transforms(
    fixed=ccfv3_img,
    moving=temp_img,
    transformlist=affine_tx['fwdtransforms'],
)

parcels_img_affine_tx = ants.apply_transforms(
    fixed=ccfv3_img, 
    moving=parcels_img, 
    transformlist=affine_tx['fwdtransforms'],
    interpolator='genericLabel',
)

In [14]:
temp_img_affine_tx

ANTsImage (ASR)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (320, 456, 528)
	 Spacing    : (0.025, 0.025, 0.025)
	 Origin     : (-13.175, -7.975, 11.375)
	 Direction  : [ 0.  0.  1.  1.  0.  0.  0. -1.  0.]

---

In [15]:
syn_tx = ants.registration(
    fixed=ccfv3_img,
    moving=temp_img_affine_tx,
    type_of_transform='SyNOnly',
    random_seed=args.SEED,
)

In [16]:
temp_img_nonlin_tx = ants.apply_transforms(
    fixed=ccfv3_img,
    moving=temp_img_affine_tx,
    transformlist=syn_tx['fwdtransforms'],
)

In [17]:
parcels_img_nonlin_tx = ants.apply_transforms(
    fixed=ccfv3_img, 
    moving=parcels_img_affine_tx,
    transformlist=syn_tx['fwdtransforms'],
    interpolator='genericLabel',
)

In [18]:
temp_img_nonlin_tx

ANTsImage (ASR)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (320, 456, 528)
	 Spacing    : (0.025, 0.025, 0.025)
	 Origin     : (-13.175, -7.975, 11.375)
	 Direction  : [ 0.  0.  1.  1.  0.  0.  0. -1.  0.]

In [19]:
parcels_img_nonlin_tx

ANTsImage (ASR)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (320, 456, 528)
	 Spacing    : (0.025, 0.025, 0.025)
	 Origin     : (-13.175, -7.975, 11.375)
	 Direction  : [ 0.  0.  1.  1.  0.  0.  0. -1.  0.]

In [22]:
np.unique(parcels_img_nonlin_tx.numpy())

array([  0.,   1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,
        11.,  12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,
        22.,  23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,
        33.,  34.,  35.,  36.,  37.,  38.,  39.,  40.,  41.,  42.,  43.,
        44.,  45.,  46.,  47.,  48.,  49.,  50.,  51.,  52.,  53.,  54.,
        55.,  56.,  57.,  58.,  59.,  60.,  61.,  62.,  63.,  64.,  65.,
        66.,  67.,  68.,  69.,  70.,  71.,  72.,  73.,  74.,  75.,  76.,
        77.,  78.,  79.,  80.,  81.,  82.,  83.,  84.,  85.,  86.,  87.,
        88.,  89.,  90.,  91.,  92.,  93.,  94.,  95.,  96.,  97.,  98.,
        99., 100., 101., 102., 103., 104., 105., 106., 107., 108., 109.,
       110., 111., 112., 113., 114., 115., 116., 117., 118., 119., 120.],
      dtype=float32)

In [20]:
TEMP_DESC = (
    f'source-{args.source}'
    f'_space-ccfv3'
    f'_res-{args.resolution}'
)

In [21]:
PARC_DESC = (
    f'source-{args.source}'
    f'_space-ccfv3'
    f'_braindiv-{args.brain_div}'
    f'_nrois-{args.num_rois}'
    f'_res-{args.resolution}'
)

In [23]:
ants.image_write(
    temp_img_nonlin_tx,
    f'{PARCELS_path}/{TEMP_DESC}_desc-template.nii.gz'
)

In [24]:
ants.image_write(
    temp_img_nonlin_tx,
    f'{PARCELS_path}/{PARC_DESC}_desc-parcels.nii.gz'
)